In [3]:
import numpy as np 

class NaiveBayes():
    
    def __init__(self, dataset):
        """
        Construtor da classe
        :param dataset: conjunto de dados
        """
        self.dados = dataset
        self.dadosPorClasse = {}
        self.resumo = {}
        self.probabilidades = {}
    
    def separarPorClasse(self):
        """
        Separa o conjunto de dados por classe. Ex {'Classe1': [[A1 A2 A3
            A4], [A1 A2 A3 A4], [A1 A2 A3 A4]]
            'Classe2': [[A1 A2 A3 A4]], ...} 
        """

        for elemento in self.dados:
            elemento = list(elemento)
            if elemento[0] not in self.dadosPorClasse:
                self.dadosPorClasse[elemento[0]] = np.array(elemento[1:])
            else:
                self.dadosPorClasse[elemento[0]] = np.vstack((self.dadosPorClasse[elemento[0]],
                elemento[1:]))
        
        return self.dadosPorClasse
    
    
    def resumir(self):
        """
        Resume os dados classificados através de dicionario com cada
        classe e a media e desvio padrao de cada um dos seus atributos
        representados pela media e o desvio padrao. Ex: {c1: [
        [mediaAtributo1, desvioPadraoAtributo1],
        [mediaAtributo2, desvioPadraoAtributo2] ], c2: ...}
        """
        self.separarPorClasse()
        
        count = 0
        for classe in self.dadosPorClasse:
            self.resumo[classe] = []
            qtd_atributos = self.dadosPorClasse[classe][0].size
            transposta = self.dadosPorClasse[classe].T
            for vetor_atributos in transposta:
                prov = [[np.mean(vetor_atributos), np.std(vetor_atributos)]]
                self.resumo[classe] += prov
        
        return self.resumo
    
    
    def calcularProbabilidadePorAtributo(self, entrada, media, desvio):
        """
        Calcula, usando a expressao da gaussiana,a probabilidade de um
        valor de atributo de entrada
        pertencer a uma classe com base no valor medio e desvio padrao desse
        atributo na classe
        """
        
        exponencial = math.exp(-(math.pow(entrada-media, 2)/
                                 (2*math.pow(desvio,2 ))))
        probabilidade = (1/(math.sqrt(2*math.pi)*desvio))*exponencial

        return probabilidade
    
    
    def calcularProbabilidades(self, elemento):
        """
        Calcula a probabilidade de um elemento de entrada pertencer a uma
        classe com base na probabilidade dos seus atributos pertencerem a classe
        """
        if not self.resumo:
            self.resumir()
            
        for classe in self.resumo:
            self.probabilidades[classe] = []
            for atributo in range(len(elemento)):
                '''Passa como parametros o valor do atributo no elemento de
                e o valor medio e desvio padrao do atributo na classe'''
                probabilidadeAtributo = self.calcularProbabilidadePorAtributo(elemento[atributo],
                    self.resumo[classe][atributo][0],
                    self.resumo[classe][atributo][1])
                if atributo<1:
                    self.probabilidades[classe] = probabilidadeAtributo
                else:
                    self.probabilidades[classe] = self.probabilidades[classe] * probabilidadeAtributo

        return self.probabilidades
    
    
    def classificar(self, elemento = None):
        """
        Retorna a classe - key - com maior probabilidade - value -
        do dicionario probabilidades
        """
        if not self.probabilidades: 
            self.calcularProbabilidades(elemento)
            
        resposta = ['classe', 0]
        for classe in self.probabilidades:
            if self.probabilidades[classe] > resposta[1]:
                resposta = [classe, self.probabilidades[classe]]

        return resposta[0]
    

In [13]:
from tabulate import tabulate
import numpy as np
import math

dataset = np.loadtxt("wine.data.txt", delimiter=',') 
alvo = [14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065]

naive = NaiveBayes(dataset)
naive.classificar(alvo)


1.0

In [14]:
total = len(dataset)
acertos = total
for i, alvo in enumerate(dataset):
    naive = NaiveBayes(np.delete(dataset,i,0))
    resposta = naive.classificar(alvo[1:])
    if resposta != alvo[0]:
        acertos -= 1

print("Precisão: ", acertos*100/total, "%")

Precisão:  97.19101123595506 %


<h3>Comparação com Naive Bayes da sklearn </h3>

In [16]:
from sklearn.naive_bayes import GaussianNB

x = dataset[:, 1:] 
y = dataset


total = len(dataset)
acertos = total
for i, alvo in enumerate(dataset):
    x = np.delete(dataset,i,0)[:, 1:] 
    y = np.delete(dataset,i,0)[:,:1]
    clf = GaussianNB()
    clf.fit(x, y)
    resposta = clf.predict(alvo[1:])
    if resposta != alvo[0]:
        acertos -= 1

print("Precisão: ", acertos*100/total, "%")


Precisão:  97.75280898876404 %


/home/hugo/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hugo/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/hugo/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hugo/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d array